<a href="https://colab.research.google.com/github/polinadrrrr/Google-Colab/blob/some-NLP/week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt

--2024-12-11 16:10:23--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-12-11 16:10:23--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0cb795818a6a8ecb56704b6481.dl.dropboxusercontent.com/cd/0/inline/CgA5aVjJxhC9oxkl83rTRZKYJIiGttFkOxkSBWMnuAbzZf8XpS2kB6juFsE72HxsMmQjkPH8RkWCSzgxrA-KyzT1gOK2vVFvoacNAXitiW17sRIiLTjAqOYPTuiR4s5MCzk/file?dl=1# [following]
--2024-12-11 16:10:24--  https://uc0cb795818a6a8ecb56704b6481.dl.dropboxusercontent.com/cd/0/inline/CgA5aVjJ

In [12]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
  data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

In [13]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [14]:
data_tok = [tokenizer.tokenize(x.lower()) for x in data]

In [15]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (string)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [16]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


In [17]:
from gensim.models import Word2Vec

model = Word2Vec(data_tok, vector_size=32, min_count=5, window=5).wv

In [18]:
model.get_vector('anything')

array([-1.3426859 , -0.46590114,  1.2579124 ,  3.4452822 ,  3.0605495 ,
        1.7698678 ,  2.1590383 , -3.5596454 , -0.42339206,  3.0226514 ,
       -1.868293  ,  1.8344376 ,  4.9094477 ,  1.099283  ,  1.8432525 ,
       -0.67014277,  0.18820232, -2.0445564 ,  0.8680008 , -2.1128309 ,
       -2.5354931 , -1.025433  , -1.9597857 , -2.4680393 ,  0.3858753 ,
       -1.3374795 ,  1.0736591 , -0.39783978,  0.4813596 ,  1.5070906 ,
       -2.2164106 ,  0.286501  ], dtype=float32)

In [19]:
model.most_similar('bread')

[('rice', 0.9464664459228516),
 ('cheese', 0.9385532140731812),
 ('corn', 0.9295622110366821),
 ('fruit', 0.9213589429855347),
 ('pasta', 0.9142268896102905),
 ('sauce', 0.9108310341835022),
 ('wine', 0.9092735052108765),
 ('beer', 0.9078968167304993),
 ('butter', 0.9065508842468262),
 ('chocolate', 0.9031502604484558)]

In [22]:
import gensim.downloader as api

model = api.load('glove-twitter-100')

In [23]:
model.most_similar(positive=["color", "money"], negative=["coin"])

[('need', 0.6363024711608887),
 ('look', 0.6207654476165771),
 ('work', 0.6155844926834106),
 ('like', 0.6001639366149902),
 ('body', 0.597007155418396),
 ('light', 0.5847621560096741),
 ('where', 0.5826524496078491),
 ('or', 0.5806817412376404),
 ('ppl', 0.5801243782043457),
 ('bad', 0.5798407196998596)]

In [24]:
words = model.index_to_key[:1000]
print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [25]:
word_vectors = np.array([model[word] for word in words])

In [26]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

In [27]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=2)
word_vectors_pca = pca.fit_transform(word_vectors)
scaler = StandardScaler()
word_vectors_pca = scaler.fit_transform(word_vectors_pca)

In [28]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "There must be a 2D vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "Points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "Points must have unit variance"

In [33]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [34]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

figure(id='p1005', ...)